In [4]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import os

# Set your OpenAI API key
# Option 1: Set it directly in the code (not recommended for production)
os.environ["OPENAI_API_KEY"] = '_____________________******'

# Option 2: Or pass it directly to the embeddings object later

# 1. Load all text files from your Git repo
REPO_DIR = r"C:\python\frappe"
file_extensions = [".py", ".md", ".txt", ".js", ".json"]

def get_file_paths(repo_dir):
    for root, _, files in os.walk(repo_dir):
        for file in files:
            if any(file.endswith(ext) for ext in file_extensions):
                yield os.path.join(root, file)

# 2. Load documents
documents = []
for filepath in get_file_paths(REPO_DIR):
    try:
        loader = TextLoader(filepath, encoding='utf-8')
        documents.extend(loader.load())
    except Exception as e:
        print(f"Failed to load {filepath}: {e}")

# 3. Chunk text using a splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100
)
chunks = splitter.split_documents(documents)

# 4. Embed chunks and store in FAISS
# Option 1: If you set the API key as an environment variable above
embeddings = OpenAIEmbeddings()

# Option 2: Or pass the API key directly
# embeddings = OpenAIEmbeddings(openai_api_key="your-api-key-here")

vectorstore = FAISS.from_documents(chunks, embeddings)

# 5. Create a retriever
retriever = vectorstore.as_retriever()

# 6. Set up the RAG pipeline
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini"),  # This also needs the API key
    retriever=retriever,
    return_source_documents=True
)

# 7. Ask a question
query = "How does authentication work in this repo?"
result = qa_chain(query)
print("Answer:")
print(result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(doc.metadata["source"])

C:\Users\i7714\AppData\Local\Temp\ipykernel_12492\107480734.py:55: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm=ChatOpenAI(model="gpt-4o-mini"),  # This also needs the API key
C:\Users\i7714\AppData\Local\Temp\ipykernel_12492\107480734.py:62: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain(query)


Answer:
The authentication in this repo allows for two-factor authentication. You can enable Two Factor Authentication from the System Settings. After enabling it, users can authenticate using their credentials along with a second factor for added security. 

For accessing the site, after setting it up, users can use the default login credentials:
- Username: Administrator
- Password: admin

Additionally, the repository mentions a script for installing dependencies, which also handles password creation for the Frappe "Administrator" user and other related users.

Sources:
C:\python\frappe\realtime\middlewares\authenticate.js
C:\python\frappe\frappe\change_log\v8\v8_8_0.md
C:\python\frappe\README.md
C:\python\frappe\frappe\email\email.md


In [8]:
query = "Where TODO docType located"
result = qa_chain(query)
print("Answer:")
print(result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(doc.metadata["source"])

Answer:
I don't know.

Sources:
C:\python\frappe\frappe\core\doctype\doctype\README.md
C:\python\frappe\frappe\custom\doctype\customize_form\customize_form.js
C:\python\frappe\frappe\desk\doctype\todo\todo_list.js
C:\python\frappe\frappe\hooks.py
